<a href="https://colab.research.google.com/github/s-yukisato/fastapi_demo/blob/main/kfold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import pickle

In [56]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
import lightgbm as lgb

In [89]:
train = pd.read_csv("../content/train.csv")
test = pd.read_csv("../content/test.csv")

In [90]:
print(f"train shape: {train.shape} | test shape : {test.shape}")

train shape: (891, 12) | test shape : (418, 11)


In [91]:
df = pd.concat([train, test])

In [92]:
df["IsAlone"] = ((df["SibSp"] == 0) & (df["Parch"] == 0)).astype(int)

In [93]:
df["Embarked"].fillna(("S"), inplace=True)

In [94]:
df["Age"].fillna((df["Age"].median()), inplace=True)
df["Fare"].fillna((df["Fare"].mean()), inplace=True)

In [95]:
df = pd.get_dummies(df, columns=["Embarked", "Sex"])

In [96]:
delete_columns = ['Name', 'PassengerId', 'Ticket', 'Cabin']
df.drop(delete_columns, axis=1, inplace=True)

In [97]:
train = df[:len(train)]
test = df[len(train):]

In [98]:
test.drop(["Survived"], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [99]:
X_train = train.drop(['Survived'], axis=1)
X_test = test
y_train = train['Survived']

In [104]:
kf = KFold(n_splits=3, shuffle=True, random_state=0)

score_list = []
models = []

for fold_, (train_ind, valid_ind) in enumerate(kf.split(X_train, y_train)):
  train_x, train_y = X_train.iloc[train_ind], y_train.iloc[train_ind]
  valid_x, valid_y = X_train.iloc[valid_ind], y_train.iloc[valid_ind]

  lgb_train = lgb.Dataset(train_x, train_y)
  lgb_valid = lgb.Dataset(valid_x, valid_y)

  lgbm_params = {
      "objective": "binary"
  }

  gbm = lgb.train(params=lgbm_params,
                  train_set=lgb_train,
                  valid_sets=[lgb_train, lgb_valid],
                  early_stopping_rounds=10,
                  verbose_eval=-1
                  )
  oof = (gbm.predict(valid_x) > 0.5).astype(int)
  score_list.append(round(accuracy_score(valid_y, oof) * 100, 2))
  models.append(gbm)

  print(f"fold{fold_ + 1} end \n")



Training until validation scores don't improve for 10 rounds.
Early stopping, best iteration is:
[38]	training's binary_logloss: 0.287729	valid_1's binary_logloss: 0.391338
fold1 end 

Training until validation scores don't improve for 10 rounds.
Early stopping, best iteration is:
[42]	training's binary_logloss: 0.285903	valid_1's binary_logloss: 0.391242
fold2 end 

Training until validation scores don't improve for 10 rounds.
Early stopping, best iteration is:
[34]	training's binary_logloss: 0.281349	valid_1's binary_logloss: 0.446937
fold3 end 



In [108]:
print(score_list, "mean_score", round(np.mean(score_list), 2))

[84.51, 84.18, 80.13] mean_score 82.94


In [113]:
score = []
for gbm in models:
  score.append(gbm.predict(train.iloc[2, :]))

print(score)
pred_2 = (np.mean(score) > 0.5).astype(int)
print(pred_2, y_train[2])

[array([0.83265794]), array([0.65960798]), array([0.89785874])]
1 1.0


/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:478: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


In [114]:
with open('model.pickle', mode="wb") as fp:
  pickle.dump(models, fp)